In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
# Importing Visitor Log Data and User_Table
DATASET_PATH = "/content/drive/MyDrive/Machine_Learning_Competition/Analytics_Vidhya_Job/"
visitor_Logs_Data = pd.read_csv(DATASET_PATH + 'VisitorLogsData.csv')
user_table = pd.read_csv(DATASET_PATH + 'userTable.csv')
input_feature_table = pd.read_csv(DATASET_PATH + "sample_submission_M7Vpb9f.csv")

print(visitor_Logs_Data.shape)
print(user_table.shape)

(6588000, 9)
(34050, 3)


In [4]:
# Display first five records of Visitor_Log_data
visitor_Logs_Data.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
0,WI10000050298,2018-05-07 04:28:45.970,pr100631,NaN,NaN,Chrome Mobile,Android,Chennai,India
1,WI10000025922,2018-05-13 07:26:04.964,pr100707,NaN,NaN,Chrome,Windows,NaN,Taiwan
2,WI100000204522,2018-05-11 11:43:42.832,pr100030,NaN,click,Chrome,windows,Gurgaon,India
3,WI10000011974,2018-05-13 15:20:23.436,Pr100192,NaN,CLICK,Chrome,Windows,,
4,WI100000441953,2018-05-08 20:44:25.238,Pr100762,NaN,click,Chrome,mac os x,Iselin,United States


In [5]:
# Display first five records of User_Table
user_table.head()

,UserID,Signup Date,User Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B
2,U109654,2013-03-19 11:38:55+00:00,B
3,U108998,2018-01-18 08:29:51.627954+00:00,C
4,U131393,2018-03-27 08:05:28.806800+00:00,B


In [6]:
# Getting Null Values in Visitor_Log_Data
visitor_Logs_Data.isnull().sum()

webClientID            0
VisitDateTime     658915
ProductID         527137
UserID           5937305
Activity          889446
Browser                0
OS                     0
City             2165831
Country           397693
dtype: int64

**Remove USER_ID which are blank**



In [7]:
# Drop Records where UserId is given as NULL
visitor_Logs_Data.dropna(axis=0, inplace=True, subset=['UserID'])
visitor_Logs_Data.shape

(650695, 9)

In [8]:
# In datetime column some dates are given in datetime format ,  Some dates are given in unix format and  some dates are given as null Values
# Separate Records with date given as NUll and Not NUll
visitor_Logs_Data_Date_not_null = visitor_Logs_Data[~visitor_Logs_Data['VisitDateTime'].isnull()]
visitor_Logs_Data_Date_null = visitor_Logs_Data[visitor_Logs_Data['VisitDateTime'].isnull()]

# Separate datetime where datetime given as datetime and datetime given in unix format
visitor_Logs_Data_1 = visitor_Logs_Data_Date_not_null[visitor_Logs_Data_Date_not_null['VisitDateTime'].str.contains('-')]
visitor_Logs_Data_2 = visitor_Logs_Data_Date_not_null[~visitor_Logs_Data_Date_not_null['VisitDateTime'].str.contains('-')]


print(visitor_Logs_Data_Date_not_null.shape)
print(visitor_Logs_Data_Date_null.shape)
print(visitor_Logs_Data_1.shape)
print(visitor_Logs_Data_2.shape)

(585892, 9)
(64803, 9)
(527141, 9)
(58751, 9)


In [9]:
# Converting Date Time from Unix Format to Datetime Format
visitor_Logs_Data_1['Visit_date_time'] = pd.to_datetime(visitor_Logs_Data_1['VisitDateTime'])
visitor_Logs_Data_2['VisitDateTime'] = visitor_Logs_Data_2['VisitDateTime'].astype('int')
visitor_Logs_Data_2['VisitDateTime'] = visitor_Logs_Data_2['VisitDateTime'] / 1000000000
visitor_Logs_Data_2['Visit_date_time'] = pd.to_datetime(visitor_Logs_Data_2['VisitDateTime'], unit='s')

In [10]:
# Concatinating all data 
visitor_Logs_Data = pd.concat([visitor_Logs_Data_1, visitor_Logs_Data_2,visitor_Logs_Data_Date_null],axis=0, sort=False).sort_index(ascending = True)
print(visitor_Logs_Data.shape)

(650695, 10)


In [11]:
# Data is given from 07-05-2018 to 27-05-2018
print("Minimum Date and Time given in dataset :", visitor_Logs_Data['Visit_date_time'].min())
print("Maximum Date and Time given in dataset :",visitor_Logs_Data['Visit_date_time'].max())

Minimum Date and Time given in dataset : 2018-05-07 00:00:01.419000
Maximum Date and Time given in dataset : 2018-05-27 23:59:23.729000


In [12]:
# Activity and ProductID values in Lower Case
visitor_Logs_Data['Activity'].replace({'CLICK':'click', 'PAGELOAD': 'pageload'}, inplace= True)
visitor_Logs_Data['ProductID'] = visitor_Logs_Data['ProductID'].str.lower() 

In [13]:
# Fill NA values in the ProductID column as per UserId and Date
visitor_Logs_Data = visitor_Logs_Data.sort_values(['UserID','Visit_date_time'])
visitor_Logs_Data['ProductID'].fillna(method='ffill', inplace=True)

In [14]:
# Fill NA values in Datetime Column as per UserID, WebClientID , ProductID
visitor_Logs_Data = visitor_Logs_Data.sort_values(['UserID','webClientID','ProductID','Visit_date_time'])
visitor_Logs_Data['Visit_date_time'].fillna(method='ffill', inplace=True)
visitor_Logs_Data.drop('VisitDateTime', axis=1, inplace=True)
visitor_Logs_Data.sort_index(inplace=True)

In [15]:
# Getting only Date, Month, Year from column Visit_date_time
visitor_Logs_Data['Visit_date'] = visitor_Logs_Data['Visit_date_time'].dt.date
visitor_Logs_Data['Visit_date'] = pd.to_datetime(visitor_Logs_Data['Visit_date'])

### Fill Values in input_feature_table

In [16]:
# Fill values in column No_of_days_Visited_7_Days
data = visitor_Logs_Data[(visitor_Logs_Data['Visit_date'] >= '2018-05-21') & (visitor_Logs_Data['Visit_date'] <= '2018-05-27')]
data_dict = dict(data.groupby(['UserID'])['Visit_date'].nunique())
input_feature_table['No_of_days_Visited_7_Days'] = input_feature_table['UserID'].map(data_dict)
input_feature_table['No_of_days_Visited_7_Days'].fillna(0, inplace=True)

In [17]:
# Fill Values in colmn No_Of_Products_Viewed_15_Days	
data = dict(visitor_Logs_Data[(visitor_Logs_Data['Visit_date'] >= '2018-05-13') & (visitor_Logs_Data['Visit_date'] <= '2018-05-27')].groupby(['UserID'])['ProductID'].nunique())
input_feature_table['No_Of_Products_Viewed_15_Days'] = input_feature_table['UserID'].map(data)
input_feature_table['No_Of_Products_Viewed_15_Days'].fillna(0, inplace=True)

In [18]:
# Fill Values in column User_Vintage
user_table['Signup Date'] = pd.to_datetime(user_table['Signup Date'])
user_table['Signup_Date'] = user_table['Signup Date'].dt.date
user_table['Signup_Date'] = pd.to_datetime(user_table['Signup_Date'])
user_table['days_as_today'] = (pd.to_datetime('2018-05-28') - user_table['Signup_Date']).dt.days

# Create dictionary
dict_1 = dict(zip(user_table['UserID'], user_table['days_as_today']))
input_feature_table['User_Vintage'] = input_feature_table['UserID'].map(dict_1)

In [19]:
# Fill Values in No_Of_Products_Viewed_15_Days
data = visitor_Logs_Data[(visitor_Logs_Data['Visit_date'] >= '2018-05-13') & (visitor_Logs_Data['Visit_date'] <= '2018-05-27') & (visitor_Logs_Data['Activity'] =='pageload')]
grouped_data = data.groupby(['UserID','ProductID'], as_index=False).size()
dict_1 = dict(zip(zip(grouped_data['UserID'], grouped_data['ProductID']), grouped_data['size']))
data['dict_column'] = list(zip(data['UserID'], data['ProductID']))
data['UserID_ProductID_Count'] = data['dict_column'].map(dict_1)
data_new = data[['UserID', 'ProductID','UserID_ProductID_Count','Visit_date_time']].sort_values(by= ['UserID','UserID_ProductID_Count','Visit_date_time'], ascending=False).reset_index(drop=True)
data_new.drop_duplicates(subset=['UserID'],keep='first', inplace=True)
#data_new['UserID_ProductID_Count'].fillna('Product101',inplace=True)
dict_2 =dict(zip(data_new['UserID'],data_new['ProductID']))
input_feature_table['Most_Viewed_product_15_Days'] = input_feature_table['UserID'].map(dict_2)
input_feature_table['Most_Viewed_product_15_Days'].fillna('Product101', inplace=True)

In [20]:
# Fill Values in column Most_Active_OS
visitor_Logs_Data['OS'] = visitor_Logs_Data['OS'].str.upper()
grouped_OS_Count = visitor_Logs_Data.groupby(['UserID','OS'], as_index=False).size()
grouped_OS_Count = grouped_OS_Count.sort_values(by=['UserID', 'size'], ascending=False).reset_index(drop=True)
grouped_OS_Count.drop_duplicates(subset='UserID', keep='first')
OS_dict = dict(zip(grouped_OS_Count['UserID'], grouped_OS_Count['OS']))
input_feature_table['Most_Active_OS'] =input_feature_table['UserID'].map(OS_dict)

In [21]:
# Fill Values in column Recently_Viewed_Product
filtered_data = visitor_Logs_Data[visitor_Logs_Data['Activity'] == 'pageload'][['UserID','Activity','Visit_date_time','ProductID']]
filtered_data = filtered_data.sort_values(by =['UserID','Visit_date_time'], ascending=False).reset_index(drop=True)
filtered_data.drop_duplicates(subset=['UserID'], keep='first', inplace=True)
product_dict = dict(zip(filtered_data['UserID'], filtered_data['ProductID']))
input_feature_table['Recently_Viewed_Product'] = input_feature_table['UserID'].map(product_dict)
input_feature_table['Recently_Viewed_Product'].fillna('Product101', inplace=True)

In [22]:
# Fill Values in column Pageloads_last_7_days
last_seven_days_data = visitor_Logs_Data[(visitor_Logs_Data['Visit_date'] >= '2018-05-21') & (visitor_Logs_Data['Visit_date'] <= '2018-05-27')]
page_load_data = last_seven_days_data[last_seven_days_data['Activity'] == 'pageload']
page_click_data = last_seven_days_data[last_seven_days_data['Activity'] == 'click']
page_load_data_dict = dict(page_load_data['UserID'].value_counts())
page_click_data_dict = dict(page_click_data['UserID'].value_counts())

input_feature_table['Pageloads_last_7_days'] = input_feature_table['UserID'].map(page_load_data_dict)
input_feature_table['Clicks_last_7_days'] = input_feature_table['UserID'].map(page_click_data_dict)

input_feature_table['Pageloads_last_7_days'].fillna(0, inplace=True)
input_feature_table['Clicks_last_7_days'].fillna(0, inplace = True)

In [23]:
input_feature_table.to_csv('/content/drive/MyDrive/Machine_Learning_Competition/Analytics_Vidhya_Job/Sample_Submission_12.csv', index=False)